In [1]:
import pandas as pd
from IPython.display import clear_output

In [2]:
xls = pd.ExcelFile('hunt4.xlsx')
days = len(xls.sheet_names)
xls.sheet_names

['12', '13', '14', '15', '16', '17', '18']

In [3]:
df1 = pd.read_excel(xls, '12')
df2 = pd.read_excel(xls, '13')
df3 = pd.read_excel(xls, '14')
df4 = pd.read_excel(xls, '15')
df5 = pd.read_excel(xls, '16')
df6 = pd.read_excel(xls, '17')
df7 = pd.read_excel(xls, '18')

frame = [df1, df2, df3, df4, df5, df6, df7]
Hunt = pd.concat(frame)
Hunt = Hunt.reset_index(drop=True)
Hunt = Hunt.rename(columns=lambda x:x.replace(' ', '_'))

In [4]:
class Guild():
    roster = []
    def __init__(self, name, total, hunt, purchased, one, two, three, four, five):
        self.name = name
        self.total = total
        self.hunt = hunt
        self.purchased = purchased
        self.one = one
        self.two = two
        self.three = three
        self.four = four
        self.five = five
        self.score = self.get_score()
        self.roster.append(self)
        
    def __repr__(self):
        return self.name       
    
    def get_score(self):
        return ((self.purchased * .3) + (self.two * .2) + (self.three * .5) + self.four + self.five)

In [5]:
# Initalize player instances
def get_roster():
    roster = []
    for _ in Hunt.Name:
        if _ not in roster:
            roster.append(_)
    for _ in roster:
        total = Hunt.query(f'Name == "{_}"').Total.sum()
        hunt = Hunt.query(f'Name == "{_}"').Hunt.sum()
        purchased = Hunt.query(f'Name == "{_}"').Purchase.sum()
        one = Hunt.query(f'Name == "{_}"').Level_1.sum()
        two = Hunt.query(f'Name == "{_}"').Level_2.sum()
        three = Hunt.query(f'Name == "{_}"').Level_3.sum()
        four = Hunt.query(f'Name == "{_}"').Level_4.sum()
        five = Hunt.query(f'Name == "{_}"').Level_5.sum()
        g = Guild(name=_, total=total, hunt=hunt, purchased=purchased, one=one, two=two, three=three, four=four, five=five)

In [6]:
# Finding the top Hunter
def top_hunter(roster):
    hunt = 0
    for _ in roster:
        if _.name != 'Total' and _.name != 'Anonymous' and _.hunt > hunt: hunt = _.hunt
    for _ in roster:
        if _.hunt == hunt:
            return _
        
# Find the top Buyer
def top_buyer(roster):
    purchases = 0
    for _ in roster:
        if _.name != 'Total' and _.name != 'Anonymous' and _.purchased > purchases: purchases = _.purchased
    for _ in roster:
        if _.purchased == purchases:
            return _
        
# Find the top Gifter
def top_gifter(roster):
    gifts = 0
    for _ in roster:
        if _.name != 'Total' and _.name != 'Anonymous' and _.total > gifts: gifts = _.total
    for _ in roster:
        if _.total == gifts: 
            return _
    
# Find the player with the top points
def top_points(roster):
    points = 0
    for _ in roster:
        if _.name != 'Total' and _.name != 'Anonymous' and _.score > points: points = _.score
    for _ in roster:
        if _.score == points:
            return _

In [7]:
# Breakdowns for categories
def breakdown(_, message):
    print(f"""{message} breakdown
    {message}: {_}
    Total Gifts: {_.total}
    Hunts: {_.hunt}
    Purchases: {_.purchased}
    Level 1s: {_.one}
    Level 2s: {_.two}
    Level 3s: {_.three}
    Level 4s: {_.four}
    Level 5s: {_.five}
    Total Points: {round(_.score, 2)}""")

# overall Breakdown
def overall_breakdown(_):
    print(f"""{_} breakdown
    Total Gifts: {_.total}
    Hunts: {_.hunt}
    Purchases: {_.purchased}
    Level 1s: {_.one}
    Level 2s: {_.two}
    Level 3s: {_.three}
    Level 4s: {_.four}
    Level 5s: {_.five}
    Total Points: {round(_.score, 2)}""")

# session average Breakdown
def session_average(_):
    print(f"""{days} day report average breakdown
    Total Gifts: {round(_.total/days, 2)}
    Hunts: {round(_.hunt/days, 2)}
    Purchases: {round(_.purchased/days, 2)}
    Level 1s: {round(_.one/days, 2)}
    Level 2s: {round(_.two/days, 2)}
    Level 3s: {round(_.three/days, 2)}
    Level 4s: {round(_.four/days, 2)}
    Level 5s: {round(_.five/days, 2)}
    Total Points: {round(_.score, 2)}""")

In [8]:
# Find a player
def find_player(player, roster):
    for _ in roster:
        if player == _.name:
            return _

In [9]:
# Finding the Naughty List
class Naughty(Guild):
    roster=[]
    
    @classmethod
    def get_naughty_list(self, days):
        print("""What it means to be on the naughty list:
    For each hunt report we recieve, we translate each level 2 hunted into .2 points, 
    level 3s into .5 points, and level 4/5 into 1 point each. 
    Purchasing a pack gives you .3 points. In total, each person is required to make 1 point daily.
    The number shown after your name is the number of total points you were missing for this hunt session.""")
        print()
        print(f"There are {len(self.roster)} players on the naughty list")
        for _ in self.roster:
            print(f"{_} is missing {round(days-_.score, 2)} days worth of completed hunts.")
        
def make_naughty_list(roster, days):
    for _ in roster:
        if _.score < days:
            Naughty(name=_.name, total=_.total, hunt=_.hunt, purchased=_.purchased, one=_.one, two=_.two, three=_.three, four=_.four, five=_.five)

In [10]:
def space_decorator():
    print()
    print("="*40)
    print()

In [11]:
def get_report():
    get_roster()
    make_naughty_list(Guild.roster, days)
    breakdown(top_hunter(Guild.roster), "Top Hunter")
    space_decorator()
    breakdown(top_buyer(Guild.roster), "Top Buyer")
    space_decorator()
    breakdown(top_gifter(Guild.roster), "Top Gifter")
    space_decorator()
    breakdown(top_points(Guild.roster), "Top Score")
    space_decorator()
    overall_breakdown(find_player("Total", Guild.roster))
    space_decorator()
    overall_breakdown(find_player("Anonymous", Guild.roster))
    space_decorator()
    session_average(find_player("Total", Guild.roster))
    space_decorator()
    Naughty.get_naughty_list(days)

In [12]:
get_report()

Top Hunter breakdown
    Top Hunter: Executive LT
    Total Gifts: 96
    Hunts: 89
    Purchases: 7
    Level 1s: 0
    Level 2s: 62
    Level 3s: 14
    Level 4s: 1
    Level 5s: 12
    Total Points: 34.5


Top Buyer breakdown
    Top Buyer: Highgarden
    Total Gifts: 106
    Hunts: 22
    Purchases: 84
    Level 1s: 0
    Level 2s: 12
    Level 3s: 3
    Level 4s: 3
    Level 5s: 4
    Total Points: 36.1


Top Gifter breakdown
    Top Gifter: Highgarden
    Total Gifts: 106
    Hunts: 22
    Purchases: 84
    Level 1s: 0
    Level 2s: 12
    Level 3s: 3
    Level 4s: 3
    Level 5s: 4
    Total Points: 36.1


Top Score breakdown
    Top Score: Highgarden
    Total Gifts: 106
    Hunts: 22
    Purchases: 84
    Level 1s: 0
    Level 2s: 12
    Level 3s: 3
    Level 4s: 3
    Level 5s: 4
    Total Points: 36.1


Total breakdown
    Total Gifts: 3097
    Hunts: 2078
    Purchases: 1019
    Level 1s: 7
    Level 2s: 1287
    Level 3s: 529
    Level 4s: 128
    Level 5s: 127
    Total P

In [16]:
x = input("set m1 noceros")

set m1 nocerosset m1 noceros


In [17]:
x

'set m1 noceros'

In [21]:
x[7:]

'noceros'

In [35]:
x = {'monsters': {
        'noceros': {
            'f2p': [
                "Level 1: :incinerator: :sage: :goblin: :elementalist: :snow:",
                "Level 2: :incinerator: :sage: :goblin: :elementalist: :snow: ",
                "Level 3: :incinerator: :sage: :goblin: :elementalist: :snow:",
                "Level 4: :incinerator: :sage: :goblin: :elementalist: :prima:",
                "Level 5: :bunny: :incinerator: :sage: :elementalist: :prima:"
            ]
        }
    }}

In [37]:
y = 'noceros'
for _ in x['monsters'][y]['f2p']:
    print(_)

Level 1: xyz
Level 2: xyz
Level 3: xyz
